# We will check if the functional dependencies we guessed are functional dependencies. If there are violations, then we will address them.

## We have already cleaned the data and populated the database. We will read in the clean data csv files that we populated the database with.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


## Checking Functional Dependencies in the Xeno Canto table

### We suspect that the Xeno Canto table is not 3NF and that we need to decompose the table. The key of Xeno Canto is id, but it would seem that genus and vernacular name are dependent only on scientific name, which would be a transitive dependency. We would guess, based on the content of the dataset, that the other attributes are dependent only on id. Let's check if these are the functional dependencies:

id -> longitude decimal, latitude decimal, country, locality, accessURI

scientific name -> genus, vernacular name

In [ ]:
# copy and paste the file path between the quotation marks below
xeno_canto_path = '/content/drive/MyDrive/CIS 4500 5500 Project/cleaned data/xeno_canto_bird_cleaned.csv'
xeno_canto = pd.read_csv(xeno_canto_path)
xeno_canto.columns

Index(['id', 'genus', 'scientificName', 'vernacularName', 'longitudeDecimal',
       'latitudeDecimal', 'country', 'locality', 'accessURI'],
      dtype='object')

We already confirmed that id is unique before cleaning the data, but let's confirm it again:

In [ ]:
len(xeno_canto['id'].unique()) == len(xeno_canto['id'])

True

Therefore whenever a row has a given id, it will always have the same values in the other columns (because there's only one such row).

Let's check if whenever a row has a given scientific name, it has the same genus.

In [ ]:
len(xeno_canto.groupby(['scientificName', 'genus'])) == len(xeno_canto['scientificName'].unique())

True

Let's check if whenever a row has a given scientific name, it has the same vernacular name.

In [ ]:
len(xeno_canto.groupby(['scientificName', 'vernacularName'])) == len(xeno_canto['scientificName'].unique())

True

We have just confirmed that scientific name -> genus, vernacular name is a functional dependency in the Xeno Canto bird relation.

As we will find out next, vernacular name -> scientific name is NOT a functional dependency in the Xeno Canto bird relation.

In [ ]:
xeno_canto.groupby(['scientificName', 'vernacularName']).size()

scientificName                        vernacularName             
Abeillia abeillei                     Emerald-chinned Hummingbird     8
Abeillia abeillei abeillei            Emerald-chinned Hummingbird     8
Abroscopus albogularis                Rufous-faced Warbler           44
Abroscopus albogularis albogularis    Rufous-faced Warbler            5
Abroscopus albogularis fulvifacies    Rufous-faced Warbler           33
                                                                     ..
Zosterornis latistriatus              Panay Striped Babbler           1
Zosterornis nigrorum                  Negros Striped Babbler          9
Zosterornis striatus                  Luzon Striped Babbler           3
Zosterornis whiteheadi                Chestnut-faced Babbler          7
Zosterornis whiteheadi ss whiteheadi  Chestnut-faced Babbler          1
Length: 24780, dtype: int64

Let's check if whenever a row has a given vernacular name, it has the same scientific name.

In [ ]:
len(xeno_canto.groupby(['scientificName', 'vernacularName'])) == len(xeno_canto['vernacularName'].unique())

False

We might also guess that country is dependent upon locality.

Let's check if whenever a row has a given locality, it has the same country

In [ ]:
len(xeno_canto.groupby(['country', 'locality'])) == len(xeno_canto['locality'].unique())

False

In [ ]:
len(xeno_canto.groupby(['country', 'locality']))

76081

In [ ]:
len(xeno_canto['locality'].unique())

75963

Country is NOT dependent upon locality.

The 'locality' column of the Xeno Canto bird table contains null values, which could explain why the rows with the same 'locality' value do not all have the same 'country' value.

In [ ]:
xeno_canto['locality'].isnull().sum()

44

In [ ]:
# remove rows with null values in 'locality' column
xeno_canto_no_null_locality = xeno_canto.dropna(axis='rows', subset='locality')
xeno_canto_no_null_locality['locality'].isnull().sum()

0

In [ ]:
len(xeno_canto_no_null_locality.groupby(['country', 'locality']))

76063

In [ ]:
len(xeno_canto_no_null_locality['locality'].unique())

75962

After dropping the null values, the number of unique locality values is still not the same as the number of unique country, locality pairs. Let's export a csv that will group the locality and country values together, first by locality, then by country, to find out what locality values have more than one country value associated with it.

In [ ]:
xeno_canto_no_null_locality.groupby(['locality', 'country']).size().to_csv('xeno_canto_no_null_locality_groupby.csv')
from google.colab import files
files.download('xeno_canto_no_null_locality_groupby.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Visual inspection of the csv shows that dropping the rows with nan as a locality value did not drop all the null locality values, as other values such as '-' and '?' were used to represent unknown or null values. Here are 2 excerpts of the csv from above:

locality|country|size

\-	Liechtenstein	4

\-	Netherlands	1

\-	Sweden	5

\--	Sweden	1

\---	Sweden	1

\----	Sweden	2



?	Argentina	8

?	Bhutan	2

?	Bolivia	35

?	Botswana	1

?	Brazil	63

?	China	3

?	Colombia	1

?	Ecuador	8

?	Finland	1

?	France	1

?	Hungary	1

?	Panama	17

?	Peru	91

?	Poland	1

?	Spain	15

?	Sweden	3

?	Taiwan	14

?	Tanzania	1

If we're going to use the 'locality' column, then we should clean these values and replace them with null. However, we're not planning on using the 'locality' column, so we can remove this column from the table.

## Checking Functional Dependencies in the World Bank Data table

We think that after the data cleaning and processing, the World Bank data has been decomposed into 3NF.

The key of the World Bank Data table is country name, indicator code, year. If we confirm that this is a unique key, then we know that the other attribute of the table, value, is dependent on the key. That is, the functional dependency is country name, indicator code, year -> value

In [ ]:
# copy and paste the file path between the quotation marks below
world_bank_data_path = '/content/drive/MyDrive/CIS 4500 5500 Project/cleaned data/world_bank_cleaned.csv'
world_bank_data = pd.read_csv(world_bank_data_path)
world_bank_data.columns

Index(['Country Name', 'Indicator Code', 'Year', 'value'], dtype='object')

In [ ]:
len(world_bank_data.groupby(['Country Name', 'Indicator Code', 'Year'])) == world_bank_data.shape[0]

True

## Checking Functional Dependencies in the World Bank Indicators table

Let's check if the indicator code is a unique key for the World Bank indicators table

In [ ]:
# copy and paste the file path between the quotation marks below
world_bank_indicators_path = '/content/drive/MyDrive/CIS 4500 5500 Project/cleaned data/world_bank_series_cleaned.csv'
world_bank_indicators = pd.read_csv(world_bank_indicators_path)
world_bank_indicators.columns

Index(['Series Code', 'Topic', 'Indicator Name', 'Long definition',
       'Unit of measure', 'Periodicity', 'Base Period', 'Other notes',
       'Aggregation method', 'Limitations and exceptions',
       'Notes from original source', 'General comments', 'Source',
       'Statistical concept and methodology', 'Development relevance'],
      dtype='object')

In [ ]:
len(world_bank_indicators['Series Code'].unique()) == len(world_bank_indicators['Series Code'])

True